# Building Predictive Models

In [1]:
# imports
import pandas as pd
import numpy as np
import sklearn
import os

## Import Data

Train and test data were randomly split within R, using 0.80 ratio.
<br>The two dataframes were written to independent csv files, and will be brought into the Python notebook now.

In [2]:
# set path to processed train/test data
processed_data_path = os.path.join(os.path.pardir, 'data', 'processed')
train_file_path = os.path.join(processed_data_path, 'train.txt')
test_file_path = os.path.join(processed_data_path, 'test.txt')

In [3]:
df_train = pd.read_csv(train_file_path, index_col='Obs')
df_test = pd.read_csv(test_file_path, index_col='Obs')

In [7]:
print('Train data:')
df_train.info()
print('\n')
print('Test data:')
df_test.info()

Train data:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 76 entries, 1 to 97
Data columns (total 9 columns):
Unnamed: 0                    76 non-null int64
Y_HighGradeCancer             76 non-null int64
PSALevel                      76 non-null float64
CancerVol                     76 non-null float64
Weight                        76 non-null float64
Age                           76 non-null float64
BenignProstaticHyperplasia    76 non-null float64
SeminalVesicleInvasion        76 non-null int64
CapsularPenetration           76 non-null float64
dtypes: float64(6), int64(3)
memory usage: 5.9 KB


Test data:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 21 entries, 5 to 95
Data columns (total 9 columns):
Unnamed: 0                    21 non-null int64
Y_HighGradeCancer             21 non-null int64
PSALevel                      21 non-null float64
CancerVol                     21 non-null float64
Weight                        21 non-null float64
Age                           2

In [8]:
df_train.columns

Index(['Unnamed: 0', 'Y_HighGradeCancer', 'PSALevel', 'CancerVol', 'Weight',
       'Age', 'BenignProstaticHyperplasia', 'SeminalVesicleInvasion',
       'CapsularPenetration'],
      dtype='object')

It looks like R appended an additional "Unnamed: 0" column, most likely related to indexing.<br>
I will remove that now.

In [9]:
# drop the redudent columns (R auto-created an index column of its own); can be seen in info() cell above
df_train = df_train.drop(columns='Unnamed: 0')
df_test = df_test.drop(columns='Unnamed: 0')

In [10]:
# examine train set
df_train.head()

,Y_HighGradeCancer,PSALevel,CancerVol,Weight,Age,BenignProstaticHyperplasia,SeminalVesicleInvasion,CapsularPenetration
Obs,,,,,,,,
1,0,-2.533700,-1.645747,-0.649508,-1.872101,-0.840562,0,-0.596573
2,0,-2.299250,-1.995368,-0.392167,-0.791989,-0.840562,0,-0.596573
3,0,-2.299250,-1.586043,-0.676493,1.368234,-0.840562,0,-0.596573
4,0,-2.299250,-2.174506,-0.416007,-0.791989,-0.840562,0,-0.596573
6,0,-1.488689,-2.046685,-0.444510,-1.872101,-0.840562,0,-0.596573


In [11]:
# examine test set
df_test.head()

,Y_HighGradeCancer,PSALevel,CancerVol,Weight,Age,BenignProstaticHyperplasia,SeminalVesicleInvasion,CapsularPenetration
Obs,,,,,,,,
5,0,-1.837148,-0.511447,-0.321415,-0.251933,-0.840562,0,-0.596573
8,0,-1.418947,-0.562625,-0.242460,-0.791989,0.706307,0,-0.596573
14,0,-0.983519,0.111131,-0.558742,0.423137,-0.840562,0,-0.596573
17,0,-0.878912,-1.509353,-0.257481,0.828178,0.305380,0,-0.450762
23,0,-0.678455,-1.611706,-0.354536,-0.656975,-0.691566,0,-0.596573


In [36]:
# create a list which captures fields to ommit from model
skip = ['Y_HighGradeCancer'
        , 'Age'
        , 'Weight'
        , 'BenignProstaticHyperplasia'
        , 'SeminalVesicleInvasion'
        , 'CapsularPenetration'
       ]
cols_model = [col for col in df_train.columns if col not in skip]
cols_model

['PSALevel', 'CancerVol']

## Data Preperation

Because R has already prepared the training and test sets, I will manually assign the split data to appropriate variables now.

In [37]:
# train-test split
X_train = df_train.loc[:, cols_model]
y_train = df_train['Y_HighGradeCancer']
X_test = df_test.loc[:, cols_model]
y_test = df_test['Y_HighGradeCancer']

In [38]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(76, 2) (76,)
(21, 2) (21,)


In [39]:
# average survival in train and test sets
print(f'Mean y in train set: {round(np.mean(y_train), 3)}')
print(f'Mean y in test set: {round(np.mean(y_test), 3)}')

Mean y in train set: 0.184
Mean y in test set: 0.333


## Baseline Model

Developing a basline model:
- Here, I will feed the dummy model training data, and sklearn will determine the most frequent classification within the Y_HighGradeCancer field (via prior analysis we know this to value to be 0). Because Y_HighGradeCancer = 0 most freqently, the model will be designed to predict 0 on every single observation.
- After the design of the baseline model, I will implement it on both the training and testing data, and calculate accuracy scores and confusion matrixes for good measure.
- Subsequent model fittings can therefore be compared to the baseline model.

In [40]:
# import function
from sklearn.dummy import DummyClassifier

In [41]:
# create model
# because mean y in train = 0.184 (shown above), this "most frequent" model will predict y=0 for all test observations
model_dummy = DummyClassifier(strategy='most_frequent', random_state=0)

In [42]:
# train model
model_dummy.fit(X_train, y_train)

DummyClassifier(constant=None, random_state=0, strategy='most_frequent')

In [43]:
# run dummy_model with training data
print(f'Score for baseline model (TRAINING): {round(model_dummy.score(X_train, y_train), 2)}')

# run dummy_model with testing data
print(f'Score for baseline model (TESTING): {round(model_dummy.score(X_test, y_test), 2)}')

Score for baseline model (TRAINING): 0.82
Score for baseline model (TESTING): 0.67


In [44]:
# performance metrics
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

In [45]:
# training confusion matrix
print(f'Confusion matrix for baseline model (TRAINING): \n {confusion_matrix(y_train, model_dummy.predict(X_train))} \n')

# testing confusion matrix
print(f'Confusion matrix for baseline model (TESTING): \n {confusion_matrix(y_test, model_dummy.predict(X_test))}')

Confusion matrix for baseline model (TRAINING): 
 [[62  0]
 [14  0]] 

Confusion matrix for baseline model (TESTING): 
 [[14  0]
 [ 7  0]]


In [31]:
# # precision and recall scores
# print(f'Precision for baseline model: {round(precision_score(y_test, model_dummy.predict(X_test)), 2)}')
# print(f'Recall for baseline model: {round(recall_score(y_test, model_dummy.predict(X_test)), 2)}')

## Logistic Regression Model

In [53]:
# import function
from sklearn.linear_model import LogisticRegression

In [54]:
# create model
model_lr_1 = LogisticRegression(random_state=0)

In [55]:
# train model
model_lr_1.fit(X_train, y_train)

C:\Users\jaosi\Anaconda3\envs\datSci\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [56]:
# evaluate model
print(f'Score for logistic regression - version 1: {round(model_lr_1.score(X_test, y_test), 4)}')

Score for logistic regression - version 1: 0.8571


In [57]:
print(f'Intercept coefficient is: {model_lr_1.intercept_}')
print(f'Slope coefficients are: {model_lr_1.coef_}')

Intercept coefficient is: [-1.95476436]
Slope coefficients are: [[0.8286437  1.00266644]]


## STATSMODELS Library

### Full Logistics Model

In [49]:
import statsmodels.api as sm

In [50]:
X_model = sm.add_constant(X_train)
model = sm.Logit(y_train, X_model)

C:\Users\jaosi\Anaconda3\envs\datSci\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [51]:
results = model.fit()

Optimization terminated successfully.
         Current function value: 0.293604
         Iterations 8


In [52]:
### full model statistical output
print(results.summary2(alpha=0.10))

                          Results: Logit
Model:              Logit             Pseudo R-squared: 0.385     
Dependent Variable: Y_HighGradeCancer AIC:              50.6278   
Date:               2020-10-28 13:55  BIC:              57.6200   
No. Observations:   76                Log-Likelihood:   -22.314   
Df Model:           2                 LL-Null:          -36.307   
Df Residuals:       73                LLR p-value:      8.3761e-07
Converged:          1.0000            Scale:            1.0000    
No. Iterations:     8.0000                                        
-------------------------------------------------------------------
                Coef.   Std.Err.     z     P>|z|    [0.05    0.95] 
-------------------------------------------------------------------
const          -2.6867    0.6186  -4.3429  0.0000  -3.7042  -1.6691
PSALevel        1.0577    0.6198   1.7067  0.0879   0.0383   2.0772
CancerVol       1.5502    0.6859   2.2599  0.0238   0.4219   2.6784

